# 정리노트 - 시계열 실습(ARIMA, SARIMA, auto_arima)

### AirPassengers

##### Box_Jenkins ARIMA Procedure
1. Data Processing
2. Identify Model to be Tentatively Entertained
3. Estimate Parameters
4. Diagnosis Check
5. User Model to Forecast

##### 1. Data processing

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv("data/AirPassengers.csv")
df.rename(columns={"Month" : "month", "#Passengers" : "passengers"}, 
          inplace=True)
df["month"] = pd.to_datetime(df["month"])
df.set_index("month", inplace=True)
# plt.figure(dpi=150)
plt.xlabel("month")
plt.ylabel("passengers")
plt.plot(df, label="passengers")
plt.legend()
display(df)

In [ ]:
import statsmodels.api as sm
#Decomposition of Time Series
decomposition = sm.tsa.seasonal_decompose(df["passengers"], 
                                          model="additive", period=1)
fig = decomposition.plot()
# fig.dpi = 150
# fig.set_size_inches(10, 15)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.2, shuffle=False)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
fig.suptitle("RawData")
sm.graphics.tsa.plot_acf(train_data.values.squeeze(), lags=30, ax=ax[0])
sm.graphics.tsa.plot_pacf(train_data.values.squeeze(), lags=30, ax=ax[1])

#비정상성 시계열의 전형적인 특징을 보여준다. 완만히 하강하는 모양새. 정상성이라면 뚝 끊기게 나올 것

In [ ]:
#차분한다.
diff_train_data = train_data.copy()
diff_train_data = diff_train_data["passengers"].diff()
diff_train_data.dropna(inplace=True)
print(diff_train_data)
# help(plt.Figure)
# plt.figure(dpi=150, figsize=(10,5))
plt.plot(diff_train_data, "orange", label="diff_train_data(Stationary)")
plt.grid()
plt.legend()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
fig.suptitle("RawData")
sm.graphics.tsa.plot_acf(diff_train_data.values.squeeze(), lags=30, ax=ax[0])
sm.graphics.tsa.plot_pacf(diff_train_data.values.squeeze(), lags=30, ax=ax[1])

#정상성을 보이는 것으로 보임. 3~4개 튀는 것들이 있으나 무시해도 좋음

##### 2. Identify Model to be Tentativery Entertained
![title](https://raw.githubusercontent.com/hrdkdh/adp-study/main/images/arima_model_selection.png)

In [ ]:
import itertools
import warnings
warnings.filterwarnings("ignore")

#원래는 위 표에 따라 p,d,q를 결정해야 하지만 iterative 하게 할거임
p = range(0, 3)
d = range(1, 2)
q = range(0, 3)
#세가지 배열의 값을 각각 조합하여 새로운 배열로 만들어 줌
pdq = list(itertools.product(p, d, q))
aic = []
for i in pdq:
    model = sm.tsa.ARIMA(train_data.values, order=(i))
    model_fit = model.fit()
    print("ARIMA Order : {} -> AIC : {}".format(i, model_fit.aic))
    aic_dict = { "order": i, "aic" : model_fit.aic }
    aic.append(aic_dict)
result_by_aic = pd.DataFrame(aic)
result_by_aic.sort_values(by="aic", ascending=True, inplace=True)
result_by_aic.reset_index(inplace=True)
result_by_aic

In [ ]:
#aic가 가장 낮은 모델로 select 하여 summary를 본다
model_opt = sm.tsa.ARIMA(train_data.values, order=result_by_aic.iloc[0,1])
model_opt_fit = model_opt.fit()
model_opt_fit.summary()

##### 3. Use Model to Forecast

In [ ]:
import datetime
import numpy as np
from sklearn.metrics import r2_score

prediction = model_opt_fit.forecast(len(test_data))
predicted_value = prediction[0]
predicted_ub = prediction[2][:,0]
predicted_lb = prediction[2][:,1]
predict_index = list(test_data.index)
r2 = r2_score(test_data, predicted_value)

forecast_start_date = datetime.datetime(1958, 8, 1)
forecast_start_date = np.datetime64(forecast_start_date)
plt.figure(dpi=150)
plt.plot(df.index, df["passengers"], label="passengers")
plt.vlines(forecast_start_date, 0, 1000, linestyle="--", color="r", label="Forecast Start")
plt.plot(predict_index, predicted_value, label = "Prediction")
plt.fill_between(predict_index, predicted_lb, predicted_ub, color="k", alpha=0.1, label="0.95 Prediction Interval")
plt.legend(loc="upper left")
plt.suptitle("ARIMA {} Prediction Results (r2_score : {:4.2f})".format(result_by_aic.iloc[0,1], r2))

##### ※SARIMA도 해보자

In [ ]:
import itertools
import warnings
warnings.filterwarnings("ignore")

p = range(0, 3)
d = range(1, 2)
q = range(0, 3)
pdq = list(itertools.product(p, d, q)) #세가지 배열의 값을 각각 조합하여 새로운 배열로 만들어 줌
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in pdq]

aic = []
for i in pdq:
    for j in seasonal_pdq:
        try: #pdq 파라미터 값 때문에 오류가 날 수 있음. 코드가 중단되므로 try문으로...
            model = sm.tsa.SARIMAX(train_data.values, order=(i), seasonal_order=(j))
            model_fit = model.fit()
            print("SARIMA Order : {}{} -> AIC : {}".format(i, j, model_fit.aic))
            aic_dict = { "pdq": i, "s-pdq" : j, "aic" : model_fit.aic }
            aic.append(aic_dict)
        except:
            continue
result_by_aic = pd.DataFrame(aic)
result_by_aic.sort_values(by="aic", ascending=True, inplace=True)
result_by_aic.reset_index(inplace=True)
result_by_aic

In [ ]:
#aic가 가장 낮은 모델로 select 하여 summary를 본다
model_opt = sm.tsa.SARIMAX(train_data.values, order=result_by_aic.iloc[0,1], 
                           seasonal_order=result_by_aic.iloc[0,2])
model_opt_fit = model_opt.fit()
model_opt_fit.summary()

In [ ]:
import datetime
import numpy as np
from sklearn.metrics import r2_score

prediction = model_opt_fit.get_forecast(len(test_data))
predicted_value = prediction.predicted_mean
predicted_ub = prediction.conf_int()[:,0]
predicted_lb = prediction.conf_int()[:,1]
predict_index = list(test_data.index)
r2 = r2_score(test_data, predicted_value)

forecast_start_date = datetime.datetime(1958, 8, 1)
forecast_start_date = np.datetime64(forecast_start_date)
plt.figure(dpi=150)
plt.plot(df.index, df["passengers"], label="passengers")
plt.vlines(forecast_start_date, 0, 1000, linestyle="--", 
           color="r", label="Forecast Start")
plt.plot(predict_index, predicted_value, label = "Prediction")
plt.fill_between(predict_index, predicted_lb, predicted_ub, color="k", 
                 alpha=0.1, label="0.95 Prediction Interval")
plt.legend(loc="upper left")
plt.suptitle("SARIMA {} {}, Prediction Results (r2_score : {:4.2f})".format(
    result_by_aic.iloc[0,1], result_by_aic.iloc[0,2], r2))

##### ※auto_arima로 SARIMA 구현

In [ ]:
#SARIMA - auto_arima로 한방에 모델 셀렉션이 가능
from pmdarima import auto_arima
aa_model = auto_arima(train_data,
                      start_p=0,
                      d=1,
                      start_q=0,
                      max_p=3,
                      max_d=3,
                      max_q=3,
                      start_P=0,
                      D=1,
                      start_Q=0,
                      max_P=3,
                      max_D=3,
                      max_Q=3,
                      m=12,
                      seasonal=True,
                      trace=True,
                      error_action="ignore",
                      suppress_warnings=True,
                      stepwise=False)
aa_model.summary()